In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from zero_helpers.imports import * 
from zero_liftsim.simmanager import SimulationManager
from zero_liftsim.helpers import base_config
from zero_liftsim.experience import AgentRideLoopExperience
from zero_liftsim.helpers import load_asset_template
from zero_liftsim import helpers as hp
from zero_liftsim.lift import Lift

import inspect

# Run Simulation

In [3]:
cfg = base_config()
cfg["SimulationManager"]["__init__"].update({"n_agents": 25, "lift_capacity": 4})
cfg['Lift']['num_chairs'] = 35

manager = SimulationManager(cfg)
result = manager.run(runtime_minutes=60*4)
print(result)

{'total_rides': 468, 'average_wait': 0.9551282051282052, 'agents': [Agent(1) rainy-news 32580404-14eb-46c7-b40b-585125c79f77, Agent(2) incompetent-sensitive d4855e94-3b32-4a41-9de5-8bfcd39f0202, Agent(3) unaccountable-example 769dae5f-5db8-4a19-9d5d-0fc57710cf00, Agent(4) electric-fishing 96a8cbd3-f06e-4921-b71f-8f07cd940046, Agent(5) modern-girlfriend 0a6fb7a3-d04b-435e-9b39-b53048c5da95, Agent(6) reminiscent-problem b470593b-8fdd-44fc-bbb3-6b718f449434, Agent(7) optimal-tower 04f661db-1235-4c95-bc98-8c22ffc4d0f1, Agent(8) nosy-closet f9403dad-e24d-4d08-910e-ffe5307b0c60, Agent(9) safe-clothes ae167d50-79bb-4275-b344-0ac1956a3561, Agent(10) squeamish-plane badb5da8-73cb-4808-b97b-09f0159a2252, Agent(11) inconclusive-history 61f63117-d218-4ab0-97dc-4bdb85ab5cf6, Agent(12) pale-currency 7ec78775-7372-4ee0-8349-71281e958ed5, Agent(13) squalid-war dbd9b555-2acb-49e5-a1ed-6eaf1ba7516c, Agent(14) truculent-dinner 609a992e-d09c-4455-989e-f2c2424dba24, Agent(15) wealthy-conversation d2ce9a34-

In [4]:
# get rideloop explogs and agent event log
exp_log_data = manager.retrieve_data()
exp = exp_log_data['exp_rideloop']
log = exp_log_data['agent_log']

## Sample Agent and Get Exp Traceback

In [5]:
def sample_agent(manager, log, exp):
    # sample agent and subset exp/log to e/l for agent subset of those
    m = log['event'] == 'ride_complete'
    event_ride_comp = log[m].sample().iloc[0].to_dict()
    agent = manager.lookup_agent(event_ride_comp['agent_uuid'])
    l = log[log['agent_uuid'] == agent.agent_uuid]
    idx = (exp['time'] - event_ride_comp['time']).dt.total_seconds().abs().idxmin()
    e = exp.loc[idx].to_dict()
    return agent, l, e

In [6]:
agent, l, e = sample_agent(manager, log, exp)

# Agent State Identification (ride loop exp)

for the ride loop agent experience, the total time is broken down into riding the lift, traversing down the mountain and waiting in the queue, i.e. "time_spent_total": ride + traverse + queue below:

sample a random time within the time range of the simulation:

In [7]:
# sample a random timestamp within the agent log timeframe
INDEX_TIME = pd.Series(pd.date_range(log['time'].min(), log['time'].max(), freq='1s')).sample().iloc[0]

In [8]:
s = inspect.getsource(AgentRideLoopExperience)
lines = s.split('\n')
idx = [i for i,x in enumerate(lines) if x.strip() == '"""'][-1]
print('\n'.join(lines[idx+1:]))

    def add_entry(self, agent, return_event_uuid, dt: datetime, ride: float, traverse: float, 
                  queue: float) -> None:
        self.log[dt] = {
            "exp_id": str(uuid()), 
            "return_event_uuid": return_event_uuid,
            "agent_uuid": agent.agent_uuid,
            "agent_uuid_codename": agent.agent_uuid_codename,
            "time_spent_ride_lift": ride,
            "time_spent_traverse_down_mountain": traverse,
            "time_spent_in_queue": queue,
            "time_spent_total": ride + traverse + queue,
        }



...and we can see in the source code above how `time_spent_total` is constructed. 

In [9]:
from zero_liftsim.agent_state_id import infer_agent_states

# func 
r = infer_agent_states([agent], INDEX_TIME)

i = agent.agent_uuid
msg = 'func infer state should return agent uuid'
assert i in r, msg
print(r)

{'68db6ced-6516-4150-8904-28f8ad57dee9': 'riding_lift'}


# Sample and Manually Review Examples

We need to evaluate how well the simulation is representing the actual constrained behavior that we see in the real world. Simply manually reviewing some examples here helps with that need. 

In [10]:
def sample_time(log):
    return pd.Series(pd.date_range(log['time'].min(), log['time'].max(), freq='1s')).sample().iloc[0]

In [11]:
time = sample_time(log)
agent, l, e = sample_agent(manager, log, exp)
agent.get_state(time)
agent.current_state

'in_queue'

In [13]:
?agent.traceback_experience

Signature: agent.traceback_experience(experience_id: 'str') -> 'str'
Docstring:
Return a formatted summary for a specific agent experience.

Retrieves the experience log matching the given ID and formats it
into a human-readable report using a Jinja2 or string template. The
report includes the logged metadata and a slice of recent agent
activity for context.

Parameters
----------
experience_id : str
    Identifier of the experience entry to summarize.

Returns
-------
str
    Rendered text describing the experience and context.

Raises
------
KeyError
    If no experience matches the provided ID.
File:      ~/code-repos/zero-lift-simulator/zero_liftsim/agent.py
Type:      method

In [26]:
exp['exp_id']

0      be65f2ce-f645-4726-9a35-c2e4540309b8
1      c20d84a7-bedc-424a-a986-441e1cfbc29b
2      6ee67af5-1a7f-4197-abb8-aa95bbc3f896
3      384fe3d6-805e-49b0-8430-6ff6bd311d6f
4      769b35fc-1da8-4a49-9b40-aa570d339457
                       ...                 
478    ade4553c-bfa9-40fb-b9cf-30c76575dc20
479    bb30f1f9-7913-4294-b0b8-afea99db1edb
480    673c8b23-542f-4998-91a2-3ce58e29cbf9
481    bb9af382-e1cc-4de7-861a-bd93e02140b5
482    53b98aac-c426-4fcf-93d3-e0472c85eaf2
Name: exp_id, Length: 483, dtype: object